In [29]:
!pip install notion-client
!pip install canvasapi
!pip install notion

In [36]:
import sys
import os
import json
from datetime import datetime
from notion_client import Client
from canvasapi import Canvas

In [37]:
# Get the absolute path to the the config file
# script_dir = os.path.dirname(os.path.abspath(__file__))
# json_file_path = os.path.join(script_dir, "config.json")
config_file_path = "/Users/marmik/Downloads/config.json"


def parse_flags() -> dict:
    """Parse the flags from the command line"""
    computer, force = False, False
    if "--computer" in sys.argv:
        computer = True
    if "--force" in sys.argv:
        force = True

    return {"computer": computer, "force": force}


def parse_config_file() -> dict:
    """Parse the config file"""
    with open(config_file_path, "r") as file:
        config = json.load(file)
    return config


def should_run_script(force_flag: bool, last_run_date_str: str) -> bool:
    """Checks conditions to determine if the script should run.
    Conditions
    - The --force flag is set
    - The script hasn't been run today, by default, the last_run_date is set to 01/01/2000
    """

    # If --force flag is provided, allow the script to run
    if force_flag:
        print("Cotion has been forced to run.")
        return True

    # Check For Timestamp
    last_run_date = datetime.strptime(last_run_date_str, "%Y-%m-%d").date()
    current_date = datetime.now().date()

    run = last_run_date != current_date

    if run:
        print(f"Cotion was last run: {last_run_date}. Running Cotion.")
    return run


def update_timestamp(config: dict):
    """Update the timestamp file with the current date."""
    current_date = datetime.now().date()
    config["LAST_RUN_DATE"] = current_date.strftime("%Y-%m-%d")
    with open(json_file_path, "w") as file:
        json.dump(config, file)


In [38]:
class Assignment:
    """A class that represents an assignment"""

    def __init__(
        self, name: str, due: str, course_name: str, assignment_id: str = ""
    ) -> None:
        self.name = name
        self.due = due
        self.course_name = course_name
        self.assignment_id = assignment_id

    def __eq__(self, other: object) -> bool:
        if isinstance(other, Assignment):
            return self.name == other.name and self.course_name == self.course_name

        return False

    def __hash__(self) -> int:
        return hash((self.name, self.due, self.course_name))

    def __str__(self) -> str:
        return f"Course: {self.course_name}, Name: {self.name}, Due: {self.due}"


In [39]:
class CanvasAPIClient:
    """A class that reads assignments in a class using the Canvas API

    Members:
    canvas: Canvas - the Canvas API client
    assignments: set - a set of assignments}
    """

    def __init__(self, domain: str, token: str) -> None:
        self.canvas = Canvas(domain, token)
        self.assignments = list()

    def read_assignments_for_course(self, course_id: int, course_name: str) -> None:
        """Given a course_id, get the assignments for that course using the Canvas API, and add them to self.assignments"""

        canvas_assignments = self.canvas.get_course(int(course_id)).get_assignments()

        for ass in canvas_assignments:
            name = ass.__getattribute__("name")

            due = (
                str(ass.__getattribute__("due_at"))[:10]
                if ass.__getattribute__("due_at")
                else None
            )

            self.assignments.append(Assignment(name, due, course_name))


In [40]:
class NotionDatabaseClient:
    """A class that uses the Notion API to interact with my Notion database

    Note:
    This code depends on the specified database having these 3 specific columns:
        Title : (Name)
        Due : (Date)
        Topic : (Select)
    """

    def __init__(self, token: str, database_id: str):
        self.notion = Client(auth=token)
        self.database_id = database_id
        self.assignments = dict()

    def read_assignments(self) -> None:
        """Use the Notion API to load in assignments from my Notion database"""
        # Notion API queries only return 100 objects at a time, and point to the next set of objects to retrieve using a "start_cursor"
        start_cursor = None

        # Start cursor will be empty on the first iteration, so we address this edge case.
        is_first_iteration = True

        while start_cursor != None or is_first_iteration:

            is_first_iteration = False
            results = self.notion.databases.query(
                self.database_id, start_cursor=start_cursor
            )

            start_cursor = results["next_cursor"]
            notion_assignments = results["results"]

            for notion_assignment in notion_assignments:

                title, due, topic, _id = None, None, None, None

                # Do not parse any assignments that do not have a title
                if len(notion_assignment["properties"]["Name"]["title"]) == 0:
                    continue
                else:
                    title = notion_assignment["properties"]["Name"]["title"][0]["text"][
                        "content"
                    ]

                if notion_assignment["properties"]["Due"]["date"]:
                    due = notion_assignment["properties"]["Due"]["date"]["start"]

                if notion_assignment["properties"]["Topic"]["select"]:
                    topic = notion_assignment["properties"]["Topic"]["select"]["name"]

                _id = notion_assignment["id"]

                assignment = Assignment(title, due, topic, _id)

                self.assignments[assignment] = assignment

    def add_new_assignment(self, assignment: Assignment) -> None:
        """Add a new assignment to the Notion database"""
        print(
            assignment.course_name,
            "New Assignment:",
            assignment.name,
            "due:",
            assignment.due,
        )
        data = {
            "Topic": {
                "type": "select",
                "select": {
                    "name": assignment.course_name,
                },
            },
            "Name": {
                "type": "title",
                "title": [{"text": {"content": assignment.name}}],
            },
        }

        if assignment.due:
            data["Due"] = {"type": "date", "date": {"start": assignment.due}}

        self.notion.pages.create(
            parent={"database_id": self.database_id}, properties=data
        )

    def update_assignment(self, page_id: str, assignment: Assignment) -> None:
        """Update an existing assignment in the notion database"""
        print(
            assignment.course_name,
            "Update Assignment:",
            assignment.name,
            "due:",
            assignment.due,
        )
        page_id_to_update = page_id

        data = {
            "Topic": {
                "type": "select",
                "select": {
                    "name": assignment.course_name,
                },
            },
            "Name": {
                "type": "title",
                "title": [{"text": {"content": assignment.name}}],
            },
            "Due": {"type": "date", "date": {"start": assignment.due}},
        }

        self.notion.pages.update(page_id=page_id_to_update, properties=data)


In [41]:
# main script
config = parse_config_file()
flags = parse_flags()

canvas_client = CanvasAPIClient(config["CANVAS"]["DOMAIN"], config["CANVAS"]["TOKEN"])
notion_client = NotionDatabaseClient(
    config["NOTION"]["TOKEN"], config["NOTION"]["DATABASE_ID"]
)

courses = config["COURSES"]


if should_run_script(
    force_flag=flags["force"], last_run_date_str=config["LAST_RUN_DATE"]
):

    for course_name, _id in courses.items():
        canvas_client.read_assignments_for_course(_id, course_name)

    notion_client.read_assignments()

    # Update / Add assignments to Notion
    for canvas_assignment in canvas_client.assignments:
        if canvas_assignment not in notion_client.assignments:
            notion_client.add_new_assignment(canvas_assignment)
        else:
            notion_assignment = notion_client.assignments[canvas_assignment]
            if (
                canvas_assignment.due
                != notion_client.assignments[canvas_assignment].due
            ):
                notion_client.update_assignment(canvas_assignment)
    update_timestamp(config)
else:
    # Only print this message if the --computer flag is not set
    if not flags["computer"]:
        print("Cotion has already been run today. Use --force to run Cotion anyway.")


Cotion was last run: 2000-01-01. Running Cotion.


ResourceDoesNotExist: Not Found